In [12]:
# HOW TO USE THIS SCRIPT
#1) in authorization put your own credentials, get them here: https://www.reddit.com/prefs/apps
#2) change keywords if work with different language
#3) change subreddit name if work with different counrty 
#4)TEST_MODE object: if true, it will collect only 200 units, if false it will collect as much as possible 

In [ ]:
#pip install praw

In [1]:
import praw #(https://praw.readthedocs.io/en/stable/index.html)
import csv
from datetime import datetime

In [3]:
#authorization 
#to use Reddit API you need to create an account and register here: https://www.reddit.com/prefs/apps

reddit = praw.Reddit(
    client_id="0mEdu6IZsH-McQZh9cbEoA",
    client_secret="abZ6QNa6scQDujsaIfyvL_6xfJIklQ",
    user_agent="siccs_project",
)

In [8]:
#set parameters 
#change keywords to another language here
keywords = [
    "Klima",
    "Erwärmung",
    "CO2",
    "erneuerbar",
    "erneuerbare Energien",
    "Hitzewelle",
    "Energie",
    "Ökologie",
    "ökologisch",
    "umweltbewusst",
    "klimabewusst",
    "ökologisch denkend",
    "umweltfreundlich",
    "klimafreundlich",
    "nachhaltig",
    "Umwelt",
    "umweltbezogen",
    "umweltpolitisch",
    "Umweltschutz",
    "Umweltprobleme",
    "Nachhaltigkeit",
    "nachhaltig",
    "CO2-Ausstoß",
    "CO2-Emissionen",
    "Meeresspiegelanstieg",
    "steigender Meeresspiegel",
    "extremes Wetter",
    "Wetterextreme",
    "Unwetterereignisse",
    "Extremwetter"
]
subreddit_name = "de"  #here change the subreddit for a country 
TEST_MODE = True  #set to False to collect ALL matching posts/comments
MAX_RECORDS = 200 if TEST_MODE else float('inf')

#storage
results = []
author_map = {}
author_counter = 1

In [9]:
#function for user anonymization
def get_anonymous_author(username):
    global author_counter
    if username is None:
        return "user_unknown"
    if username not in author_map:
        author_map[username] = f"user_{author_counter}"
        author_counter += 1
    return author_map[username]

In [10]:
#main collecting lopp
subreddit = reddit.subreddit(subreddit_name)

#collecting data
query = " OR ".join(keywords)

for submission in subreddit.search(query, sort="new", time_filter="all", limit=None):
    if len(results) >= MAX_RECORDS:
        break

    #check if any keyword appears in title or body of text
    if any(kw.lower() in (submission.title + " " + submission.selftext).lower() for kw in keywords):
        results.append({
            "type": "post",
            "keyword": next((kw for kw in keywords if kw.lower() in (submission.title + " " + submission.selftext).lower()), ""),
            "post_id": submission.id,
            "post_title": submission.title,
            "post_body": submission.selftext if submission.selftext.strip() else submission.url,
            "post_upvotes": submission.score,
            "post_upvote_ratio": submission.upvote_ratio,
            "post_num_comments": submission.num_comments,
            "post_created_utc": submission.created_utc,
            "post_url": submission.url,
            "post_subreddit": submission.subreddit.display_name,
            "post_author_anon": get_anonymous_author(str(submission.author)),
            "comment_id": "",
            "comment_body": "",
            "comment_upvotes": "",
            "comment_created_utc": "",
            "comment_author_anon": ""
        })

    if len(results) >= MAX_RECORDS:
        break

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list(): 
        if len(results) >= MAX_RECORDS:
            break

        if any(kw.lower() in comment.body.lower() for kw in keywords):
            results.append({
                "type": "comment",
                "keyword": next((kw for kw in keywords if kw.lower() in comment.body.lower()), ""),
                "post_id": submission.id,
                "post_title": submission.title,
                "post_body": "",
                "post_upvotes": "",
                "post_upvote_ratio": "",
                "post_num_comments": "",
                "post_created_utc": "",
                "post_url": submission.url,
                "post_subreddit": submission.subreddit.display_name,
                "post_author_anon": "",
                "comment_id": comment.id,
                "comment_body": comment.body,
                "comment_upvotes": comment.score,
                "comment_created_utc": comment.created_utc,
                "comment_author_anon": get_anonymous_author(str(comment.author))
            })


In [11]:
#save to csv
filename = "reddit_test_dataset.csv" if TEST_MODE else "reddit_full_dataset.csv"
with open(filename, "w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

print(f"Saved {len(results)} records to {filename}")

Saved 200 records to reddit_test_dataset.csv
